#Published on November 26, 2022 (17:50 GMT) by Marília Prata, mpwolke

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#Where is the Canarinho Pistol? Where is Brazil’s invoked mascot walking?

How the Canarinho Pistola was born

"The Canarinho Pistola became the official mascot of Brazil in 2016. His frown and pouty manners endeared him to Brazilians and he became an obligatory figure in the games."

![](https://news.trenddetail.com/temp/resized/medium_2022-11-24-fab941e62c.jpg)
https://news.trenddetail.com/euro/253133.html

In [ ]:
!pip install "docarray[full]==0.4.4" -q

In [ ]:
!pip install torchvision==0.11.2 -q

In [ ]:
import os
DATA_DIR = "/kaggle/input/fifa-2022-all-players-image-dataset/Images/Images/Group G/Brazil Players"
DATA_PATH = f"{DATA_DIR}/*/*.jpg"
MAX_DOCS = 1000
os.listdir(DATA_DIR)
#os.listdir(DATA_DIR)[:10]

In [ ]:
from docarray import DocumentArray, Document

In [ ]:
# Use `.from_files` to quickly load them into a `DocumentArray`
docs = DocumentArray.from_files(DATA_PATH, size=MAX_DOCS)
print(f"{len(docs)} Documents in DocumentArray")

In [ ]:
docs.plot_image_sprites()

In [ ]:
from docarray import Document

# Convert to tensor, normalize so they're all similar enough
def preproc(d: Document):
    return (d.load_uri_to_image_tensor()  # load
             .set_image_tensor_shape((80, 60))  # ensure all images right size (dataset image size _should_ be (80, 60))
             .set_image_tensor_normalization()  # normalize color 
             .set_image_tensor_channel_axis(-1, 0))  # switch color axis for the PyTorch model later

In [ ]:
# apply en masse
docs.apply(preproc)

In [ ]:
import torchvision
model = torchvision.models.resnet50(pretrained=True)  # load ResNet50

In [ ]:
# Use GPU if available
import torch
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
    
docs.embed(model, device=device)

In [ ]:
DATA_DIR = "/kaggle/input/fifa-2022-all-players-image-dataset/Images/Images/Group G/Brazil Players"
DATA_PATH = f"{DATA_DIR}/Images_Richarlison/*.jpg"

print(os.listdir("/kaggle/input/fifa-2022-all-players-image-dataset/Images/Images/Group G/Brazil Players/")[10:20])
os.listdir(DATA_DIR)[:10]

#Number Nine: Richarlison image.

In [ ]:
QUERY_IMAGE = "/kaggle/input/fifa-2022-all-players-image-dataset/Images/Images/Group G/Brazil Players/Images_Richarlison/Richarlison12.jpg"
query_doc = Document(uri=QUERY_IMAGE)
query_doc.display()

In [ ]:
# Throw the one Document into a DocumentArray, since that's what we're matching against
query_docs = DocumentArray([query_doc])

# Apply same preprocessing
query_docs.apply(preproc)

# ...and create embedding just like we did with the dataset
query_docs.embed(model, device=device) # If running on non-gpu machine, change "cuda" to "cpu"

query_docs.match(docs, limit=4)

DocumentArray(query_doc.matches, copy=True).apply(lambda d: d.set_image_tensor_channel_axis(0, -1).set_image_tensor_inv_normalization())

# for doc in DocumentArray(query_doc.matches, copy=True).apply(lambda d: d.set_image_tensor_channel_axis(0, -1).set_image_tensor_inv_normalization()):
#     doc.display()

(DocumentArray(query_doc.matches, copy=True)
    .apply(lambda d: d.set_image_tensor_channel_axis(0, -1)
                      .set_image_tensor_inv_normalization())).plot_image_sprites()

In [ ]:
!pip install gradio

In [ ]:
def get_similar_images(image_url):
    
    !wget {image_url} -O mpwolke.jpg

    query_doc = Document(uri='/kaggle/working/mpwolke.jpg')
    
    # Throw the one Document into a DocumentArray, since that's what we're matching against
    query_docs = DocumentArray([query_doc])

    # Apply same preprocessing
    query_docs.apply(preproc)

    # ...and create embedding just like we did with the dataset
    query_docs.embed(model, device=device) # If running on non-gpu machine, change "cuda" to "cpu"

    query_docs.match(docs, limit=4)

    DocumentArray(query_doc.matches, copy=True).apply(lambda d: d.set_image_tensor_channel_axis(0, -1).set_image_tensor_inv_normalization())

    for doc in DocumentArray(query_doc.matches, copy=True).apply(lambda d: d.set_image_tensor_channel_axis(0, -1).set_image_tensor_inv_normalization()):
        doc.display()

    return (DocumentArray(query_doc.matches, copy=True)
        .apply(lambda d: d.set_image_tensor_channel_axis(0, -1)
                          .set_image_tensor_inv_normalization())).plot_image_sprites()

In [ ]:
get_similar_images("https://english.cdn.zeenews.com/sites/default/files/2022/05/10/1040825-rohit-sharma-mi.jpg")

In [ ]:
import numpy as np

import gradio as gr


demo = gr.Interface(get_similar_images, 'text', "image")

demo.launch(share=True)

#Einstein tongue style

In [ ]:
from docarray import DocumentArray

da_png = DocumentArray.from_files('../input/fifa-2022-all-players-image-dataset/Images/Images/Group G/Brazil Players/Images_Richarlison/Richarlison1.jpg')
da_png.plot_image_sprites()

#https://docarray.jina.ai/fundamentals/documentarray/construct/

#Acknowledgements:

Afiz https://www.kaggle.com/code/afiz23/jina-doc-array